In [99]:
from typing import Type, get_origin, get_args
from pydantic import BaseModel

def generate_field_description(model: Type[BaseModel], indent: int = 0) -> str:
    """
    Recursively generates a formatted field description and a JSON example
    for a Pydantic model, including nested classes.
    """
    indent_str = '  ' * indent
    field_descriptions = []
    json_example_lines = []

    for field_name, field_info in model.model_fields.items():
        annotation = field_info.annotation
        field_desc = field_info.description or ""

        # Detect if it's a nested Pydantic model
        if isinstance(annotation, type) and issubclass(annotation, BaseModel):
            field_descriptions.append(
                f"{indent_str}- {field_name} [object]: {field_desc}"
            )
            # Recurse for nested model
            nested_desc, nested_json = generate_field_description_with_json(annotation, indent + 1)
            field_descriptions.append(nested_desc)
            json_example_lines.append(f'{indent_str}"{field_name}": {{\n{nested_json}\n{indent_str}}}')
        else:
            type_name = annotation.__name__ if hasattr(annotation, "__name__") else str(annotation)
            field_descriptions.append(f"{indent_str}- {field_name} [{type_name}]: {field_desc}")
            json_example_lines.append(f'{indent_str}"{field_name}": <{type_name}>')

    field_description_str = "\n".join(field_descriptions)
    json_example_str = ",\n".join(json_example_lines)

    if indent == 0:
        # Top-level output
        return (
            "Generate a JSON response. The fields should include:\n" +
            field_description_str +
            "\n\nPlease provide the next response **only** in the following valid JSON format, without any additional text:\n\n" +
            "{\n" + json_example_str + "\n}"
        )
    else:
        # Return components for recursive case
        return field_description_str, json_example_str

def generate_field_description_with_json(model: Type[BaseModel], indent: int = 0):
    """
    Helper function that returns both the nested field description and nested JSON.
    """
    return generate_field_description(model, indent)

In [100]:
from pydantic import BaseModel, Field

class Topic(BaseModel):  # Inherit from BaseModel
    topic: str = Field(..., description="The topic")
    topic_reason: str = Field(..., description="Reasoning for choosing the topic for the given input, explain what is in the input and how it is related to the topic.")

class row_topic(BaseModel):
    topic1: Topic = Field(..., description="Best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the most relavant topic to the input out of the given list.")
    topic2: Topic = Field(..., description="Second best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the second relavant topic to the input out of the given list.")
    topic3: Topic = Field(..., description="Third best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the third relavant topic to the input out of the given list.")





In [101]:
print(generate_field_description(row_topic))

Generate a JSON response. The fields should include:
- topic1 [object]: Best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the most relavant topic to the input out of the given list.
  - topic [str]: The topic
  - topic_reason [str]: Reasoning for choosing the topic for the given input, explain what is in the input and how it is related to the topic.
- topic2 [object]: Second best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the second relavant topic to the input out of the given list.
  - topic [str]: The topic
  - topic_reason [str]: Reasoning for choosing the topic for the given input, explain what is in the input and how it is related to the topic.
- topic3 [object]: Third best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the third relavant topic to the input out of the given list.
  - topic

In [102]:
import requests

# Define the server URL (update if needed)
SERVER_URL = "http://localhost:5000"

try:
    # Send a GET request
    response = requests.get(SERVER_URL+'/health')

    # Check if request succeeded (HTTP status code 200)
    if response.status_code == 200:
        print(f"Success: Server responded with {response.status_code}")
        print("Response Content:", response.text)
    else:
        print(f"Warning: Server responded with status code {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error: Could not connect to the server.\nDetails: {e}")

Success: Server responded with 200
Response Content: {"status":"running"}



In [103]:
# Corrected candidate topics list 
candidate_topics = [
    "Auschwitz",
    "Baltic states and Sweden",
    "Boder trying to understand reasons, skepticism",
    "Christians, church, religion",
    "France, French",
    "Germany, Germans",
    "Interactions with SS and SS actions",
    "Interactions with and feelings about Germans, Aryan identities",
    "Italy",
    "Jews and Jewish identity, false papers",
    "Jews and fate of Jewish community",
    "Palestine, Judaism, kibbutzim",
    "Paris, France",
    "Poland, Polish",
    "Russia, Russians",
    "Switzerland",
    "To and from cities and countries",
    "Actions, choices, and intentions",
    "Arrest, beating",
    "Behavior, treatment, and aftermath",
    "Birth, living situation, camp life",
    "Children",
    "Clarifications, mostly geographic and people",
    "Closed-ended, factual questions, mostly yes-no questions",
    "Clothing and material objects",
    "Collective actions vs. individual agency",
    "Countryside (forest, fields, and farms)",
    "Days, daily life, daily routines",
    "Encounters with Germans",
    "Escaping, leaving, being evacuated, fleeing",
    "Factory work, forced labor, business",
    "Factual clarification (name, address, number, meaning)",
    "Family separation",
    "Family (background, what happened to)",
    "Food, eating, and drinking",
    "Fuel",
    "Geographic locations, camp names/locations",
    "Ghetto experiences",
    "Hospital, doctor",
    "How long (lengths of time)",
    "How many (people)",
    "How many people in family",
    "How old (age)",
    "Hygiene, toilet facilities, body violations",
    "Introductions, storytelling",
    "Killing, shooting, and death",
    "Lager (names, conditions, factual details)",
    "Liberation, end of war",
    "Living quarters (homes, barracks)",
    "Locations and activities in streets, cities",
    "Longer questions about unique Holocaust experiences",
    "Marriage, marital relations, family",
    "Men (fathers, sons, brothers, husbands, uncles)",
    "Men and women",
    "Military, soldiers, and construction",
    "Mistreatment and death, corpses, gassings, crematoria",
    "Mistreatment and reactions",
    "Music, song",
    "Night",
    "Numerical quantities (mostly people)",
    "Numerical quantities and dates",
    "Outbreak of the war, events of war",
    "Payments and wages",
    "Prisoners",
    "Professional work and industry",
    "Quantities (size, weight, amounts, distance)",
    "Questions about men",
    "Questions of meaning",
    "References to women",
    "Schooling",
    "Sexual violence and assault",
    "Short negation questions (factual)",
    "Short questions about processes",
    "Shorter durations of time (hours, days, weeks, months)",
    "Sickness, disease, illness",
    "Sleeping arrangements, beds",
    "Smoking, cigarettes, bartering",
    "Studying, learning",
    "Tattoos",
    "Tell me/describe",
    "Time (month, year), when",
    "Transportation and deportation",
    "War destruction",
    "What",
    "What happened (next)",
    "What, how",
    "Where",
    "Who",
    "Why, how",
    "Witnessing events",
    "Women (mothers, daughters, sisters, wives, aunts)",
    "Writing/language/inscriptions"
]

In [ ]:
import requests
import json


FLASK_API_URL = "http://127.0.0.1:5000/chat"

def generate_payload(question,answer):
    # Define the request payload (matches the curl command)
    payload = {
        "model": "DeepSeek-R1:latest",
        "stream": False,
        "messages": [
            {
                "role": "system",
                "content": (
                generate_field_description(row_topic)
                ),
            },
            {
                "role": "system",
                "content": "You are informed about the Holocaust during World War II. You are a helpful assistant tasked with tagging conversations realated to the Holocaust based on their content."
            },
            {
                "role": "system",
                "content": "these are the topics you can choose from: " + str(candidate_topics)
            },
            {
                "role": "user",
                "content": f'''
                    The interview question is: {question}
                    The given response and answer is: {answer}
                '''
            }
        ]
    }
    return payload


def content_request(payload):
# Set the headers for JSON request
    headers = {"Content-Type": "application/json"}

    try:
        # Send POST request to the Flask API
        response = requests.post(FLASK_API_URL, headers=headers, json=payload)

        # Check if the request was successful
        if response.status_code == 200:
            response_json = response.json()
            raw_content = response_json["message"]["content"]
            #print(raw_content)
            return raw_content

        else:
            print(f"Error {response.status_code}: {response.text}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")



def content_request2(payload):
    
    


In [105]:
temp = json.loads('''{
"topic1": {
  "topic": "Interactions with SS and SS actions",
  "topic_reason": "The speaker mentions that their father was a prisoner in World War I taken by the Germans, which implies interaction with the German authorities."
},
"topic2": {
  "topic": "Actions of Nazis during World War II",
  "topic_reason": "Although not explicitly stated, the mention of Dora's family being affected by the Germans' actions during World War I and their subsequent birth in Germany implies involvement of Nazi policies and practices."
},
"topic3": {
  "topic": "Family separation due to war",
  "topic_reason": "The speaker describes a situation where her parents were separated from each other due to German captivity, which resulted in the birth of two children. This suggests a family has been torn apart by war-related circumstances."
}
                  }''')

print(type(temp))



import json

valid_json_str = '''
{
  "topic1": {
    "topic": "Test Topic",
    "topic_reason": "Because it fits"
  }
}
'''

parsed = json.loads(valid_json_str)
print(type(parsed))  # Should output: <class 'dict'>



<class 'dict'>
<class 'dict'>


In [106]:
print(generate_field_description(row_topic))


Generate a JSON response. The fields should include:
- topic1 [object]: Best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the most relavant topic to the input out of the given list.
  - topic [str]: The topic
  - topic_reason [str]: Reasoning for choosing the topic for the given input, explain what is in the input and how it is related to the topic.
- topic2 [object]: Second best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the second relavant topic to the input out of the given list.
  - topic [str]: The topic
  - topic_reason [str]: Reasoning for choosing the topic for the given input, explain what is in the input and how it is related to the topic.
- topic3 [object]: Third best fit topic out of the provided topic list, make sure it is different from the other two topics. This should be the third relavant topic to the input out of the given list.
  - topic

In [107]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
import json


# Point to the folder path
input_folder = Path("Phase2_Folders/030_processed_html")

output_folder= "/Users/xiyuancao/Desktop/USHMM-PDF-cleansing/Gender_family_topics_identification/topic_labeling"
# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

class File:
    def __init__(self, file_path):
        self.file_path = file_path
        self.topics = {}


# Dictionary to store results
files = {}


def safe_json_loads(s):
    """Attempts to fix and parse a possibly incomplete JSON string."""
    s = s.strip()
    
    # Basic brace balancing
    open_braces = s.count('{')
    close_braces = s.count('}')
    
    if close_braces < open_braces:
        s += '}' * (open_braces - close_braces)
    
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed: {e}")
        return None

def single_csv_classification(file_obj, output_path):
    """Process a single CSV file and classify topics for each row."""
    df = pd.read_csv(file_obj.file_path)
    df["topics"] = ""

    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            question = row["Question"]
            answer = row["Answer"]
            #print(question+"\n"+answer)
            payload = generate_payload(question, answer)
            #print(payload)
            topics = content_request(payload)
            #topics=[1,2,3]
            df.at[i, "topics"] = topics
            
            print(topics)
            
            temp = safe_json_loads(topics)
            if temp:
                # Extract only the topic strings (e.g., temp["topic1"]["topic"])
                for key in temp:
                    if isinstance(temp[key], dict) and "topic" in temp[key]:
                        topic_str = temp[key]["topic"]
                        file_obj.topics[topic_str] = file_obj.topics.get(topic_str, 0) + 1

            

            # Count topic frequencies
            for key, value in temp.items():
                print(f"{key}: {value}")
                #file_obj.topics[key] = file_obj.topics.get(topic, 0) + 1

        except Exception as e:
            print(f"Error at row {i}: {e}")
            df.at[i, "topics"] = []

    # Save to file
    filename = Path(file_obj.file_path).stem
    df.to_csv(f"{output_path}/{filename}_labeled.csv", index=False)



n=1
# Loop through all CSV files in the folder
for file_path in input_folder.glob("*.csv"):
    if(n==1):
        break 
    n=n+1

    file_obj = File(file_path)
    single_csv_classification(file_obj, output_folder)
    files[file_path.name] = file_obj  # Save the object to access topic counts later


file_obj = File("/Users/xiyuancao/Desktop/USHMM-PDF-cleansing/Phase2_Folders/030_processed_html/RG-50.030.0038_trs_en_cleaned.csv")
single_csv_classification(file_obj, output_folder)
files[file_path.name] = file_obj  # Save the object to access topic counts later


# Example: Print topic counts from all files
for fname, file_obj in list(files.items())[:1]:
    print(f"\n {fname}")
    #print(file_obj.topics)
    for topic, count in file_obj.topics.items():
        print(f"  {topic}: {count}")

  1%|▏         | 1/75 [00:43<54:10, 43.93s/it]

<think>
Okay, so the user has provided a JSON structure with three topic slots. They want me to fill these based on an interview response about someone's name and birthplace. The topics available are a bunch of historical ones related to the Holocaust.

First, looking at the response: "My name is Leo Bretholz. Do I have to spell that? I usually do have to. And I was born in Vienna, Austria." This seems like personal information about someone's name and place of birth. 

I need to figure out which topics this fits into. The first topic should be the most relevant. Since the user asked for a name and birthplace, it directly relates to "Biography or life story" but that's not in the provided list. Wait, let me check the available topics again.

Looking through the list: 'Jews and Jewish identity...' could be related if Leo Bretholz was Jewish, which I'm assuming given the context of Holocaust-related conversations. But without knowing for sure, it might be a stretch. Alternatively, "Where

  3%|▎         | 2/75 [00:53<28:58, 23.81s/it]

<think>
Okay, let me try to figure this out. So the user has provided some data about Holocaust education and a JSON response based on an interview question asking for the year when something happened. Specifically, they mentioned that a witness named Anna survived Auschwitz in 1943.

Now, I need to create another JSON response following the same structure but with different topics. The task is to generate a response where topic1 is different from the previous one and then include two more relevant topics.

First, looking at the previous example: topic1 was about Auschwitz, which made sense because it's directly related to the event. So for this new question about concentration camps in general, topic1 should be something like "Concentration camps (general)" or another broad category that includes all types of camps used by Nazi Germany.

Next, topic2 and topic3 need to cover other relevant areas. Since the user has a long list of topics, I can pick another area, maybe "Numbers" becaus

  4%|▍         | 3/75 [01:07<22:50, 19.04s/it]

...

<think>
Alright, let's tackle this query step by step. The user has provided a JSON structure that includes three topics related to the Holocaust and has asked for a response tagging their conversation based on these topics.

First, I need to understand the context. The interview question is about where someone went, specifically mentioning Trier in Germany. This falls under 'Germany, Germans' as it's directly referring to a location associated with the Holocaust.

Next, I should consider why this location might be significant. Trier was part of the German territories that were occupied and subjected to forced labor during the Nazi era. This makes it highly relevant to the topic of 'Germany, Germans'.

The third topic needs to be something else but still related. Perhaps a nearby place like Koblenz could make sense here since it's also part of the occupied region and was used in concentration camps.

I should ensure that each topic is distinct from the others, so I'll assign different 

 13%|█▎        | 10/75 [02:11<09:46,  9.02s/it]

<think>
Okay, so I need to create a JSON response based on this user's query. Let me read through the entire text first to understand what it's about.

The user provided a detailed narrative starting with a request for a JSON response in the form of a story about a young man escaping Nazi Germany and making his way to Israel. It mentions various steps like moving from Luxembourg to Belgium, crossing into France, and ending up in Baltimore. There are also references to events like Kristallnacht.

Hmm, the user's query seems to be asking for a JSON response based on this narrative but it doesn't explicitly state what exactly they want in the JSON format. They just wrote "Create a JSON response" after providing the story. I need to figure out what information should be included in the JSON.

Looking at the structure of typical JSON responses, it's usually an array or an object containing key-value pairs. Since this is a narrative with multiple entities like names, places, events, etc., pe

 13%|█▎        | 10/75 [02:11<14:17, 13.19s/it]


KeyboardInterrupt: 